# Detección de anomalías

En algunos casos, los datos pueden contener valores codificados como valores que pueden parecer válidos pero que son erroneos. Pueden provenir de errores en el proceso de medición o algún otro proceso que se haya hecho con los datos. Por ejemplo, la grabación de datos manual es bastante propensa a la introducción de errores.

En general, la mejor estrategia será tener una captación de datos de calidad que nos asegure que no haya valores erroneos. Cuando esto no sea posible, podemos intentar detectar algunos errores que son más evidentes por su magnitud y, ademas, también pueden estropear los modelos precisamente por forzarles a ajustarse a unos valores de una magnitud completamente fuera de lo normal. Estos valores atípicos pueden ser relativamente fáciles de identificar. Otros estarán al límite y nos pueden hacer dudar. Por otra parte, puede haber valores atípicos que parezcan normales en el rango de su característica pero que no encajen con los valores de las demás características. Estos podríamos detectarlos con alguna técnica multidimensional.

Para una buena definición de lo que es un valor atípico (*outlier*) se puede leer esta sección: [What are outliers in the data?](https://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm)

**Lectura recomendada**: Este [artículo resume muy bien el problema y los principales métodos de detección de outliers](https://towardsdatascience.com/a-brief-overview-of-outlier-detection-techniques-1e0b2c19e561).


## z-score

Aquí nos centraremos en el método uni-variable z-score. Otros métodos multivariable tiene más sentido probarlos después de haber estudiado otros modelos de aprendizaje automático. El z-score de un valor se calcula con (donde $x$ es el valor, $\mu$ es la media y $\sigma$ es la desviación estándar):

$$z = \frac{|x-\mu|}{\sigma}$$

Como ejemplo, vamos a aplicarlo sobre un conjunto de datos de salud.

In [6]:
from sklearn import datasets
dataset = datasets.fetch_openml(name='plasma_retinol', version=2, as_frame=True)
tabla = dataset.frame
tabla

,AGE,SEX,SMOKSTAT,QUETELET,VITUSE,CALORIES,FAT,FIBER,ALCOHOL,CHOLESTEROL,BETADIET,RETDIET,BETAPLASMA,binaryClass
0,64.0,Female,Former,21.48380,Yes_fairly_often,1298.8,57.0,6.3,0.0,170.3,1945.0,890.0,200.0,N
1,76.0,Female,Never,23.87631,Yes_fairly_often,1032.5,50.1,15.8,0.0,75.8,2653.0,451.0,124.0,N
2,38.0,Female,Former,20.01080,Yes_not_often,2372.3,83.6,19.1,14.1,257.9,6321.0,660.0,328.0,N
3,40.0,Female,Former,25.14062,No,2449.5,97.5,26.5,0.5,332.6,1061.0,864.0,153.0,N
4,72.0,Female,Never,20.98504,Yes_fairly_often,1952.1,82.6,16.2,0.0,170.8,2863.0,1209.0,92.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,46.0,Female,Former,25.89669,No,2263.6,98.2,19.4,2.6,306.5,2572.0,1261.0,164.0,P
311,45.0,Female,Never,23.82703,Yes_fairly_often,1841.1,84.2,14.1,2.2,257.7,1665.0,465.0,80.0,P
312,49.0,Female,Never,24.26126,Yes_fairly_often,1125.6,44.8,11.9,4.0,150.5,6943.0,520.0,300.0,P
313,31.0,Female,Former,23.45255,Yes_fairly_often,2729.6,144.4,13.2,2.2,381.8,741.0,644.0,121.0,N


Necesitamos la media y la desviación típica de cada característica. Para hacerlo de una forma cómoda, rápida y eficiente (calcularlas solo una vez), vamos a usar el método `describe` de los objetos `Dataframe` de Pandas.

In [7]:
desc = tabla.describe()
desc

,AGE,QUETELET,CALORIES,FAT,FIBER,ALCOHOL,CHOLESTEROL,BETADIET,RETDIET,BETAPLASMA
count,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000
mean,50.146032,26.157374,1796.654603,77.033333,12.788571,3.279365,242.460635,2185.603175,832.714286,189.892063
std,14.575226,6.013550,680.347435,33.829443,5.330192,12.322880,131.991614,1473.886547,589.289030,183.000803
min,19.000000,16.331140,445.200000,14.400000,3.100000,0.000000,37.700000,214.000000,30.000000,0.000000
25%,39.000000,21.799715,1338.000000,53.950000,9.150000,0.000000,155.000000,1116.000000,480.000000,90.000000
50%,48.000000,24.735250,1666.800000,72.900000,12.100000,0.300000,206.300000,1802.000000,707.000000,140.000000
75%,62.500000,28.853415,2100.450000,95.250000,15.600000,3.200000,308.850000,2836.000000,1037.000000,230.000000
max,83.000000,50.403330,6662.200000,235.900000,36.800000,203.000000,900.700000,9642.000000,6901.000000,1415.000000


Usando esa información podemos calcular fácilmente el z-score de cada valor numérico de la tabla y anotar las filas que contienen algún valor que se pasa de un umbral. El siguiente código muestra como hacerlo. La parte más complicada es la impresión para comprobar lo que estamos haciendo. Con Pandas, el cálculo del z-score y anotar las filas es bastante sencillo.

In [8]:
umbral_z_score = 4.0

# Cabecera
print('    ', end='')
for caracteristica in desc:
    print('{:>8}'.format(caracteristica[:8]), end=' ')
print()

# Calcular z-score, anotando filas con atipicos y mostrando sus valores
filas_atipicos = []
for i, fila in tabla.iterrows():
    print('{:<4d}'.format(i), end='')
    for caract in desc:
        print('{:>8.2f}'.format(tabla.loc[i][caract]), end=' ')
    print('\n    ', end='')

    atipico_detectado = False
    for caract in desc:
        z_score = abs(tabla.loc[i][caract] - desc.loc['mean'][caract]) / desc.loc['std'][caract]
        if z_score > umbral_z_score:
            atipico_detectado = True
        print('z {:>6.2f}'.format(z_score), end=' ')
    print()
    if atipico_detectado:
        filas_atipicos.append(i)

         AGE QUETELET CALORIES      FAT    FIBER  ALCOHOL CHOLESTE BETADIET  RETDIET BETAPLAS 
0      64.00    21.48  1298.80    57.00     6.30     0.00   170.30  1945.00   890.00   200.00 
    z   0.95 z   0.78 z   0.73 z   0.59 z   1.22 z   0.27 z   0.55 z   0.16 z   0.10 z   0.06 
1      76.00    23.88  1032.50    50.10    15.80     0.00    75.80  2653.00   451.00   124.00 
    z   1.77 z   0.38 z   1.12 z   0.80 z   0.56 z   0.27 z   1.26 z   0.32 z   0.65 z   0.36 
2      38.00    20.01  2372.30    83.60    19.10    14.10   257.90  6321.00   660.00   328.00 
    z   0.83 z   1.02 z   0.85 z   0.19 z   1.18 z   0.88 z   0.12 z   2.81 z   0.29 z   0.75 
3      40.00    25.14  2449.50    97.50    26.50     0.50   332.60  1061.00   864.00   153.00 
    z   0.70 z   0.17 z   0.96 z   0.60 z   2.57 z   0.23 z   0.68 z   0.76 z   0.05 z   0.20 
4      72.00    20.99  1952.10    82.60    16.20     0.00   170.80  2863.00  1209.00    92.00 
    z   1.50 z   0.86 z   0.23 z   0.16 z   0.64 z

Por tanto, las filas con algún valor con un z-score por encima del umbral (4 si no se ha cambiado) son las siguientes. Podemos ver en el cuadro anterior los valores y ver como la fila 39 se debe al valor de BETAPLASMA, la 50 al de FIBER, en la 61 a los de CALORIAS y ALCOHOL y así sucesivamente. Interesará echarles un vistazo para ver si realmente son valores que no tienen sentido o, si considerando las propiedades medidas por cada característica, pueden ser razonables.

El método z-score está basado en la distribución normal y, por tanto, en características que sigan distribuciones muy diferentes puede ser necesario aplicar algún otro método más específico.

In [9]:
filas_atipicos

[39, 50, 61, 93, 151, 170, 207, 218, 225, 256, 261, 262, 308]

Puede ayudarnos a tener una visión más clara de lo que estamos haciendo añadir dos filas a la tabla descriptiva indicando los límites. Así el límite lo vemos en valores de la característica. Para el umbral de 4 se consideran atípicos los que sobrepasan los valores:

In [10]:
desc.loc['4-score'] = [desc.loc['mean'][caract] + 4*desc.loc['std'][caract] for caract in desc]
desc.loc['-4-score'] = [desc.loc['mean'][caract] - 4*desc.loc['std'][caract] for caract in desc]
desc

,AGE,QUETELET,CALORIES,FAT,FIBER,ALCOHOL,CHOLESTEROL,BETADIET,RETDIET,BETAPLASMA
count,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000
mean,50.146032,26.157374,1796.654603,77.033333,12.788571,3.279365,242.460635,2185.603175,832.714286,189.892063
std,14.575226,6.013550,680.347435,33.829443,5.330192,12.322880,131.991614,1473.886547,589.289030,183.000803
min,19.000000,16.331140,445.200000,14.400000,3.100000,0.000000,37.700000,214.000000,30.000000,0.000000
25%,39.000000,21.799715,1338.000000,53.950000,9.150000,0.000000,155.000000,1116.000000,480.000000,90.000000
50%,48.000000,24.735250,1666.800000,72.900000,12.100000,0.300000,206.300000,1802.000000,707.000000,140.000000
75%,62.500000,28.853415,2100.450000,95.250000,15.600000,3.200000,308.850000,2836.000000,1037.000000,230.000000
max,83.000000,50.403330,6662.200000,235.900000,36.800000,203.000000,900.700000,9642.000000,6901.000000,1415.000000
4-score,108.446935,50.211575,4518.044343,212.351105,34.109341,52.570884,770.427091,8081.149361,3189.870408,921.895277
-4-score,-8.154871,2.103172,-924.735136,-58.284438,-8.532198,-46.012153,-285.505821,-3709.943012,-1524.441836,-542.111150


**Ejercicio**: Utiliza esta técnica para identificar el outlier del conjunto de datos 'cps_85_wages' del repositorio OpenML.